# Sujet
L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.
La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :
•	Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.
•	Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.
•	Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.
Le direction fait donc appel à vous, spécialistes de l'analyse de données, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.


# Etape 1 : Préparation des données

In [16]:
# Import libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Import csv files
in_time_csv = pd.read_csv('./Base Data/in_time.csv')
out_time_csv = pd.read_csv('./Base Data/out_time.csv')
general_data_csv = pd.read_csv('./Base Data/general_data.csv')
manager_survey_data_csv = pd.read_csv('./Base Data/manager_survey_data.csv')
employee_survey_data_csv = pd.read_csv('./Base Data/employee_survey_data.csv')

Préparation des données de pointage :
- On supprime les valeurs manquantes (NaN) qui correspondent à un non-pointage
- On supprime les colonnes composées uniquement de NaN
- On transforme les pointages en quantité horaires travaillées

In [18]:
# Process in and out times
in_time_csv.shape
#in_time_csv.head

(4410, 262)

In [19]:
# Append the two csv files, convert all fields except the header and the first two columns to pandas datetime
total_time_csv = in_time_csv.append(out_time_csv, ignore_index=True)
total_time_csv = total_time_csv.replace(np.nan, 0)
total_time_csv.iloc[:, 1:] = total_time_csv.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')
total_time_csv = total_time_csv.diff(periods=4410, axis=0)
total_time_csv = total_time_csv.iloc[4410:]
total_time_csv.reset_index(inplace=True)
total_time_csv.drop(columns=['Unnamed: 0', '2015-01-01', '2015-01-14','2015-01-26','2015-03-05',
             '2015-05-01','2015-07-17','2015-09-17','2015-10-02',
              '2015-11-09','2015-11-10','2015-11-11','2015-12-25', 'index'
             ],axis=1,inplace=True)
total_time_csv['Mean Time']=total_time_csv.mean(axis=1)
total_time_csv['hrs']=total_time_csv['Mean Time']/np.timedelta64(1, 'h')
total_time_csv.reset_index(inplace=True)
total_time_csv.drop(total_time_csv.columns.difference(['index','hrs']), 1, inplace=True)
total_time_csv.rename(columns={'index': 'EmployeeID'},inplace=True)

total_time_csv

,EmployeeID,hrs
0,0,6.870229
1,1,7.315971
2,2,6.816081
3,3,6.789215
4,4,7.877561
...,...,...
4405,4405,8.316921
4406,4406,5.897197
4407,4407,7.149526
4408,4408,9.187612


In [20]:
# Merge the multiple csv files
complete_csv = pd.merge(general_data_csv, total_time_csv, on='EmployeeID', how='inner')
complete_csv = pd.merge(complete_csv, manager_survey_data_csv, on='EmployeeID', how='inner')
complete_csv = pd.merge(complete_csv, employee_survey_data_csv, on='EmployeeID', how='inner')
complete_csv

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,hrs,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,6,1,0,0,7.315971,3,3,3.0,4.0,2.0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,3,5,1,4,6.816081,2,4,3.0,2.0,4.0
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,2,5,0,3,6.789215,3,3,2.0,2.0,1.0
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,5,8,7,5,7.877561,2,3,4.0,4.0,3.0
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,2,6,0,4,10.275802,3,3,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,29,No,Travel_Rarely,Sales,4,3,Other,1,4405,Female,...,2,6,1,5,8.316921,2,3,3.0,4.0,3.0
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,...,5,3,0,2,5.897197,3,3,4.0,1.0,3.0
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,...,2,3,0,2,7.149526,2,3,4.0,4.0,3.0
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,...,4,4,1,2,9.187612,3,4,1.0,3.0,3.0
